In [18]:
import requests
from io import BytesIO, StringIO
from io import BytesIO
from zipfile import ZipFile
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,8.27)})
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import joblib
import os.path
import tensorflow as tf
np.random.seed(0)
tf.random.set_seed(0)
FROM_SCRATCH = False
TF_MODEL_FNAME = 'tf-clf-wine'
RFC_FNAME = 'rfc-wine'
ENC_FNAME = 'wine_encoder'
DEC_FNAME = 'wine_decoder'
TF_MODEL_FNAMER = 'tf-clf-sales'
RFC_FNAMER = 'rfc-sales'
ENC_FNAMER = 'sales_encoderR'
DEC_FNAMER = 'sales_decoderR'

9.1.1 Preparing the data.
We’re using the wine-quality dataset, a numeric tabular dataset containing features that refer to the chemical composition of wines and quality ratings. To make this a simple classification task, we bucket all wines with ratings greater
than five as good, and the rest we label bad. We also normalize all the features.

In [19]:
def fetch_wine_ds():
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
    resp = requests.get(url, timeout=2)
    resp.raise_for_status()
    string_io = StringIO(resp.content.decode('utf-8'))
    return pd.read_csv(string_io, sep=';')

In [20]:
store = pd.read_csv('input/store.csv')
train = pd.read_csv('input/train.csv',parse_dates=[2])
test = pd.read_csv('input/test.csv',parse_dates=[3])
# fillna in store with 0 has better result than median()
# Aufbereiten der daten
store.fillna(0, inplace=True)
# fill missing values in test with 1
# Aufbereiten der Daten
test.fillna(value = 1, inplace = True)
# merge data with store
# Alles in eine Tabelle
train = pd.merge(train, store, on='Store')
test = pd.merge(test, store, on='Store')


C:\Users\Christian Karg\AppData\Local\Temp\ipykernel_2216\2235936024.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('input/train.csv',parse_dates=[2])


In [21]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0


In [22]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,1,4,2015-09-17,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,857,1,3,2015-09-16,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
2,1713,1,2,2015-09-15,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
3,2569,1,1,2015-09-14,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
4,3425,1,7,2015-09-13,0.0,0,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0


In [23]:
# process train and test
# aufbereiten der daten, neue Spalten und manche werden entfernt
def process(data, isTest = False):
    # label encode some features
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    # buchstaben zu zahlen
    data.StoreType.replace(mappings, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.StateHoliday.replace(mappings, inplace=True)

    # extract some features from date column
    data['Month'] = data.Date.dt.month
    data['Year'] = data.Date.dt.year
    data['Day'] = data.Date.dt.day
    data['WeekOfYear'] = data.Date.dt.weekofyear

    # calculate competiter open time in months
    data['CompetitionOpen'] = 12 * (data.Year - data.CompetitionOpenSinceYear) + \
        (data.Month - data.CompetitionOpenSinceMonth)
    data['CompetitionOpen'] = data['CompetitionOpen'].apply(lambda x: x if x > 0 else 0)

    # calculate promo2 open time in months
    data['PromoOpen'] = 12 * (data.Year - data.Promo2SinceYear) + \
        (data.WeekOfYear - data.Promo2SinceWeek) / 4.0
    data['PromoOpen'] = data['PromoOpen'].apply(lambda x: x if x > 0 else 0)

    # Indicate whether the month is in promo interval
    month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', \
             7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
    data['month_str'] = data.Month.map(month2str)

    def check(row):
        if isinstance(row['PromoInterval'],str) and row['month_str'] in row['PromoInterval']:
            return 1
        else:
            return 0

    data['IsPromoMonth'] =  data.apply(lambda row: check(row),axis=1)

    # select the features we need
    features = ['Store', 'DayOfWeek', 'Promo', 'StateHoliday', 'SchoolHoliday',
       'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'Year', 'Month', 'Day',
       'WeekOfYear', 'CompetitionOpen', 'PromoOpen', 'IsPromoMonth']
    if not isTest:
        features.append('Sales')

    data = data[features]
    return data

train = train.sort_values(['Date'],ascending = False)
train = process(train)
test = process(test,isTest = True)

C:\Users\Christian Karg\AppData\Local\Temp\ipykernel_2216\2713858278.py:15: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data['WeekOfYear'] = data.Date.dt.weekofyear
C:\Users\Christian Karg\AppData\Local\Temp\ipykernel_2216\2713858278.py:15: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data['WeekOfYear'] = data.Date.dt.weekofyear


In [24]:
test.head()

,Store,DayOfWeek,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,Year,Month,Day,WeekOfYear,CompetitionOpen,PromoOpen,IsPromoMonth
0,1,4,1,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,2015,9,17,38,84.0,24189.50,0
1,1,3,1,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,2015,9,16,38,84.0,24189.50,0
2,1,2,1,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,2015,9,15,38,84.0,24189.50,0
3,1,1,1,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,2015,9,14,38,84.0,24189.50,0
4,1,7,0,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,2015,9,13,37,84.0,24189.25,0


In [25]:
train.head()

,Store,DayOfWeek,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,...,Promo2SinceWeek,Promo2SinceYear,Year,Month,Day,WeekOfYear,CompetitionOpen,PromoOpen,IsPromoMonth,Sales
0,1,5,1,0,1,3,1,1270.0,9.0,2008.0,...,0.0,0.0,2015,7,31,31,82.0,24187.75,0,5263
679364,747,5,1,0,1,3,3,45740.0,8.0,2008.0,...,0.0,0.0,2015,7,31,31,83.0,24187.75,0,10708
702362,772,5,1,0,1,4,3,1850.0,0.0,0.0,...,0.0,0.0,2015,7,31,31,24187.0,24187.75,0,5224
683890,752,5,1,0,1,1,1,970.0,3.0,2013.0,...,31.0,2013.0,2015,7,31,31,28.0,24.00,0,7763
17714,20,5,1,0,0,4,1,2340.0,5.0,2009.0,...,40.0,2014.0,2015,7,31,31,74.0,9.75,1,9593


In [26]:
df = fetch_wine_ds()
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [27]:
train['class'] = 'bad'
train.loc[(train['Sales'] > 6300), 'class'] = 'good'
featuresR  = ['Store', 'DayOfWeek', 'Promo', 'StateHoliday', 'SchoolHoliday',
       'StoreType', 'Assortment', 'CompetitionDistance',
       'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear', 'Promo2',
       'Promo2SinceWeek', 'Promo2SinceYear', 'Year', 'Month', 'Day',
       'WeekOfYear', 'CompetitionOpen', 'PromoOpen', 'IsPromoMonth']
train['good'] = 0
train['bad'] = 0
train.loc[train['class'] == 'good', 'good'] = 1
train.loc[train['class'] == 'bad', 'bad'] = 1
train_data = train[featuresR].to_numpy()
labels_train = train[['class','good', 'bad']].to_numpy()

X_trainR, X_testR, y_trainR, y_testR = train_test_split(train_data, labels_train, random_state=0)
X_trainR, X_testR = X_trainR.astype('float32'), X_testR.astype('float32')
y_train_labR, y_test_labR = y_trainR[:, 0], y_testR[:, 0]
y_trainR, y_testR = y_trainR[:, 1:].astype('float32'), y_testR[:, 1:].astype('float32')
scalerR = StandardScaler()
scalerR.fit(X_trainR)

StandardScaler()

In [28]:
train.head()

,Store,DayOfWeek,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,...,Month,Day,WeekOfYear,CompetitionOpen,PromoOpen,IsPromoMonth,Sales,class,good,bad
0,1,5,1,0,1,3,1,1270.0,9.0,2008.0,...,7,31,31,82.0,24187.75,0,5263,bad,0,1
679364,747,5,1,0,1,3,3,45740.0,8.0,2008.0,...,7,31,31,83.0,24187.75,0,10708,good,1,0
702362,772,5,1,0,1,4,3,1850.0,0.0,0.0,...,7,31,31,24187.0,24187.75,0,5224,bad,0,1
683890,752,5,1,0,1,1,1,970.0,3.0,2013.0,...,7,31,31,28.0,24.00,0,7763,good,1,0
17714,20,5,1,0,0,4,1,2340.0,5.0,2009.0,...,7,31,31,74.0,9.75,1,9593,good,1,0


In [29]:
print(X_trainR)

[[8.320000e+02 5.000000e+00 1.000000e+00 ... 2.416800e+04 4.900000e+01
  0.000000e+00]
 [4.070000e+02 1.000000e+00 0.000000e+00 ... 1.180000e+02 2.900000e+01
  1.000000e+00]
 [3.980000e+02 7.000000e+00 0.000000e+00 ... 2.417300e+04 2.875000e+01
  0.000000e+00]
 ...
 [5.850000e+02 5.000000e+00 1.000000e+00 ... 2.000000e+00 2.417375e+04
  0.000000e+00]
 [7.420000e+02 6.000000e+00 0.000000e+00 ... 2.417800e+04 2.417850e+04
  0.000000e+00]
 [1.043000e+03 2.000000e+00 0.000000e+00 ... 8.200000e+01 2.415725e+04
  0.000000e+00]]


In [30]:
df['class'] = 'bad'
df.loc[(df['quality'] > 5), 'class'] = 'good'
features = [
    'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
    'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
    'pH', 'sulphates', 'alcohol'
    ]
df['good'] = 0
df['bad'] = 0
df.loc[df['class'] == 'good', 'good'] = 1
df.loc[df['class'] == 'bad', 'bad'] = 1
data = df[features].to_numpy()
labels = df[['class','good', 'bad']].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(data, labels, random_state=0)
X_train, X_test = X_train.astype('float32'), X_test.astype('float32')
y_train_lab, y_test_lab = y_train[:, 0], y_test[:, 0]
y_train, y_test = y_train[:, 1:].astype('float32'), y_test[:, 1:].astype('float32')
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

Select good wine instance
We partition the dataset into good and bad portions and select an instance of interest. I’ve chosen it to be a good quality
wine.
Note that bad wines are class 1 and correspond to the second model output being high, whereas good wines are class
0 and correspond to the first model output being high.

In [31]:
bad_days = np.array([a for a, b in zip(X_trainR, y_trainR) if b[1] == 1])
good_days = np.array([a for a, b in zip(X_trainR, y_trainR) if b[1] == 0])
xR = np.array([[747,5,1,0,1,3,3,45740.0,8.0,2008.0,1,0.0,0.0,2015,7,31,31,83.0,24187.75,0]])

In [32]:
bad_wines = np.array([a for a, b in zip(X_train, y_train) if b[1] == 1])
good_wines = np.array([a for a, b in zip(X_train, y_train) if b[1] == 0])
x = np.array([[9.2, 0.36, 0.34, 1.6, 0.062, 5., 12., 0.99667, 3.2, 0.67, 10.5]]) #prechosen instance

9.1.2 Training models
Creating an Autoencoder
For some of the explainers, we need an autoencoder to check whether example instances are close to the training data
distribution or not.

In [33]:
from tensorflow.keras.layers import Dense
from tensorflow import keras
ENCODING_DIM = 3 #7
BATCH_SIZE = 32 #64
EPOCHS = 50 #100
class AER(keras.Model):
    def __init__(self, encoder: keras.Model, decoder: keras.Model, **kwargs) -> None:
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
    def call(self, x: tf.Tensor, **kwargs):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat
def make_aeR():
    len_input_output = X_trainR.shape[-1]
    encoder = keras.Sequential()
    encoder.add(Dense(units=ENCODING_DIM*2, activation="relu", input_shape=(len_input_output)))
    encoder.add(Dense(units=ENCODING_DIM, activation="relu"))
    decoder = keras.Sequential()
    decoder.add(Dense(units=ENCODING_DIM*2, activation="relu", input_shape=(ENCODING_DIM)))
    decoder.add(Dense(units=len_input_output, activation="linear"))
    ae = AER(encoder=encoder, decoder=decoder)
    ae.compile(optimizer='adam', loss='mean_squared_error')
    history = ae.fit(
        scalerR.transform(X_trainR),
        scalerR.transform(X_trainR),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        verbose=False,)
    # loss = history.history['loss']
    # plt.plot(loss)
    # plt.xlabel('Epoch')
    # plt.ylabel('MSE-Loss')
    ae.encoder.save(f'{ENC_FNAMER}.h5')
    ae.decoder.save(f'{DEC_FNAMER}.h5')
    return ae

def load_ae_modelR():
    encoder = load_model(f'{ENC_FNAMER}.h5')
    decoder = load_model(f'{DEC_FNAMER}.h5')
    return AER(encoder=encoder, decoder=decoder)

In [34]:
from tensorflow.keras.layers import Dense
from tensorflow import keras
ENCODING_DIM = 7
BATCH_SIZE = 64
EPOCHS = 100
class AE(keras.Model):
    def __init__(self, encoder: keras.Model, decoder: keras.Model, **kwargs) -> None:
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
    def call(self, x: tf.Tensor, **kwargs):
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat
def make_ae():
    len_input_output = X_train.shape[-1]
    encoder = keras.Sequential()
    encoder.add(Dense(units=ENCODING_DIM*2, activation="relu", input_shape=(len_input_output)))
    encoder.add(Dense(units=ENCODING_DIM, activation="relu"))
    decoder = keras.Sequential()
    decoder.add(Dense(units=ENCODING_DIM*2, activation="relu", input_shape=(ENCODING_DIM)))
    decoder.add(Dense(units=len_input_output, activation="linear"))
    ae = AE(encoder=encoder, decoder=decoder)
    ae.compile(optimizer='adam', loss='mean_squared_error')
    history = ae.fit(
        scaler.transform(X_train),
        scaler.transform(X_train),
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        verbose=False,)
    # loss = history.history['loss']
    # plt.plot(loss)
    # plt.xlabel('Epoch')
    # plt.ylabel('MSE-Loss')
    ae.encoder.save(f'{ENC_FNAME}.h5')
    ae.decoder.save(f'{DEC_FNAME}.h5')
    return ae

def load_ae_model():
    encoder = load_model(f'{ENC_FNAME}.h5')
    decoder = load_model(f'{DEC_FNAME}.h5')
    return AE(encoder=encoder, decoder=decoder)


Random Forest Model
We need a tree-based model to get results for the tree SHAP explainer. Hence we train a random forest on the winequality dataset.

In [35]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

def make_rfcR():
    rfc = RandomForestClassifier(n_estimators=50)
    rfc.fit(scalerR.transform(X_trainR), y_train_labR)
    y_predR = rfc.predict(scalerR.transform(X_testR))
    print('accuracy_score:', accuracy_score(y_predR, y_test_labR))
    print('f1_score:', f1_score(y_test_labR, y_predR, average=None))
    joblib.dump(rfc, f"{RFC_FNAMER}.joblib")
    return rfc

def load_rfc_modelR():
    return joblib.load(f"{RFC_FNAMER}.joblib")

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

def make_rfc():
    rfc = RandomForestClassifier(n_estimators=50)
    rfc.fit(scaler.transform(X_train), y_train_lab)
    y_pred = rfc.predict(scaler.transform(X_test))
    print('accuracy_score:', accuracy_score(y_pred, y_test_lab))
    print('f1_score:', f1_score(y_test_lab, y_pred, average=None))
    joblib.dump(rfc, f"{RFC_FNAME}.joblib")
    return rfc

def load_rfc_model():
    return joblib.load(f"{RFC_FNAME}.joblib")


Tensorflow Model
Finally, we also train a TensorFlow model.


In [37]:
from keras.models import load_model

In [38]:
from tensorflow import keras
from tensorflow.keras import layers

def make_tf_modelR():
    inputs = keras.Input(shape=X_trainR.shape[1])
    x = layers.Dense(6, activation="relu")(inputs)
    outputs = layers.Dense(2, activation="softmax")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])
    history = model.fit(
        scalerR.transform(X_trainR),
        y_trainR,
        epochs=15,
        verbose=False,
        validation_data=(scalerR.transform(X_testR), y_testR),
        )
    y_predR = model(scalerR.transform(X_testR)).numpy().argmax(axis=1)
    print('accuracy_score:', accuracy_score(y_predR, y_testR.argmax(axis=1)))
    print('f1_score:', f1_score(y_predR, y_testR.argmax(axis=1), average=None))
    model.save(f'{TF_MODEL_FNAMER}.h5')
    return model
def load_tf_modelR():
    return load_model(f'{TF_MODEL_FNAMER}.h5')

In [39]:
from tensorflow import keras
from tensorflow.keras import layers

def make_tf_model():
    inputs = keras.Input(shape=X_train.shape[1])
    x = layers.Dense(6, activation="relu")(inputs)
    outputs = layers.Dense(2, activation="softmax")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])
    history = model.fit(
        scaler.transform(X_train),
        y_train,
        epochs=30,
        verbose=False,
        validation_data=(scaler.transform(X_test), y_test),
        )
    y_pred = model(scaler.transform(X_test)).numpy().argmax(axis=1)
    print('accuracy_score:', accuracy_score(y_pred, y_test.argmax(axis=1)))
    print('f1_score:', f1_score(y_pred, y_test.argmax(axis=1), average=None))
    model.save(f'{TF_MODEL_FNAME}.h5')
    return model
def load_tf_model():
    return load_model(f'{TF_MODEL_FNAME}.h5')

Load/Make models
We save and load the same models each time to ensure stable results. If they don’t exist we create new ones. If you
want to generate new models on each notebook run, then set FROM_SCRATCH=True.

In [40]:

modelR = make_tf_modelR()
rfcR = make_rfcR()
aeR = make_aeR()


accuracy_score: 0.742264149459503
f1_score: [0.68409206 0.7823439 ]
accuracy_score: 0.9245231082606182
f1_score: [0.93494089 0.91013288]


TypeError: 'int' object is not iterable

In [41]:
#if FROM_SCRATCH or not os.path.isfile(f'{TF_MODEL_FNAME}.h5'):
#    model = make_tf_model()
#    rfc = make_rfc()
#    ae = make_ae()
#else:
#    rfc = load_rfc_model()
#    model = load_tf_model()
#    ae = load_ae_model()
model = make_tf_model()
rfc = make_rfc()
ae = make_ae()

accuracy_score: 0.7425
f1_score: [0.75534442 0.72823219]
accuracy_score: 0.805
f1_score: [0.78212291 0.82352941]


TypeError: 'int' object is not iterable

9.1.3 Util functions
These are utility functions for exploring results. The first shows two instances of the data side by side and compares
the difference. We’ll use this to see how the counterfactuals differ from their original instances. The second function
plots the importance of each feature. This will be useful for visualizing the attribution methods

In [43]:
def compare_instances(x, cf):
    """
    Show the difference in values between two instances.
    """
    x = x.astype('float64')
    cf = cf.astype('float64')
    for f, v1, v2 in zip(features, x[0], cf[0]):
        print(f'{f:<25} instance: {round(v1, 3):^10} counter factual: {round(v2, 3):^10} difference: {round(v2, 7):^5}')

def plot_importance(feat_imp, feat_names, class_idx, **kwargs):
    """
    Create a horizontal barchart of feature effects, sorted by their magnitude.
    """
    df = pd.DataFrame(data=feat_imp, columns=feat_names).sort_values(by=0, axis='columns')
    feat_imp, feat_names = df.values[0], df.columns
    fig, ax = plt.subplots(figsize=(10, 5))
    y_pos = np.arange(len(feat_imp))
    ax.barh(y_pos, feat_imp)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(feat_names, fontsize=15)
    ax.invert_yaxis()
    ax.set_xlabel(f'Feature effects for class {class_idx}', fontsize=15)
    return ax, fig

9.1.5 Local Necessary Features
Anchors
Anchors tell us what features need to stay the same for a specific instance for the model to give the same classification.
In the case of a trained image classification model, an anchor for a given instance would be a minimal subset of the
image that the model uses to make its decision.
Here we apply Anchors to the tensor flow model trained on the wine-quality dataset.

In [44]:
from alibi.explainers import AnchorTabular
predict_fn = lambda x: model.predict(scaler.transform(x))
explainer = AnchorTabular(predict_fn, features)
explainer.fit(X_train, disc_perc=(25, 50, 75))
result = explainer.explain(x, threshold=0.95)

4/4 [==============================] - 0s 2ms/step


In [47]:
predict_fnR = lambda x: modelR.predict(scalerR.transform(x))
explainerR = AnchorTabular(predict_fnR, featuresR)
explainerR.fit(X_trainR, disc_perc=(25, 50, 75))
resultR = explainerR.explain(xR, threshold=0.95)

4/4 [==============================] - 0s 1ms/step


In [48]:
print('Anchor =', result.data['anchor'])
print('Precision = ', result.data['precision'])
print('Coverage = ', result.data['coverage'])

Anchor = ['volatile acidity <= 0.39', 'alcohol > 10.10']
Precision =  0.9796437659033079
Coverage =  0.16263552960800667


In [49]:
print('Anchor =', resultR.data['anchor'])
print('Precision = ', resultR.data['precision'])
print('Coverage = ', resultR.data['coverage'])

Anchor = ['Promo > 0.00', 'SchoolHoliday > 0.00', 'DayOfWeek <= 6.00', 'StateHoliday <= 0.00', 'PromoOpen > 24171.75']
Precision =  0.9955947136563876
Coverage =  0.023005455455849082


In [50]:
idx = 0
#class_names = adult.target_names
print(explainer.predictor(X_test[idx].reshape(1, -1))[0])
#print('Prediction: ', class_names[explainer.predictor(X_test[idx].reshape(1, -1))[0]])

1/1 [==============================] - 0s 23ms/step
1


In [51]:
idx = 11
print(explainer.predictor(X_test[idx].reshape(1, -1))[0])

1/1 [==============================] - 0s 24ms/step
1


In [52]:
explanation = explainer.explain(X_test[idx], threshold=0.95)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

4/4 [==============================] - 0s 2ms/step
Anchor: volatile acidity > 0.64 AND alcohol <= 11.00 AND sulphates <= 0.62 AND residual sugar <= 2.20
Precision: 0.96
Coverage: 0.08


In [53]:
print(explainerR.predictor(X_testR[idx].reshape(1, -1))[0])
explanation = explainerR.explain(X_testR[idx], threshold=0.95)
print('Anchor: %s' % (' AND '.join(explanation.anchor)))
print('Precision: %.2f' % explanation.precision)
print('Coverage: %.2f' % explanation.coverage)

1/1 [==============================] - 0s 23ms/step
1
4/4 [==============================] - 0s 1ms/step
Anchor: Promo <= 0.00 AND WeekOfYear <= 22.00 AND SchoolHoliday <= 0.00 AND StoreType > 1.00
Precision: 0.97
Coverage: 0.13
